In [1]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np

In [2]:
# Import the datasets 
senseData = pd.read_csv("Ltrain.csv", header = None)
senseData.columns = ["Case_ID", "Timestamp", "Type", "Sensor"]
senseData

,Case_ID,Timestamp,Type,Sensor
0,2022-09-02,2022-09-02T08:03:44.275852343,PresenceSensor,door_A
1,2022-09-02,2022-09-02T08:03:49.518493029,FloorSensor,sens3_A
2,2022-09-02,2022-09-02T08:04:19.518493029,FloorSensor,sens3_A
3,2022-09-02,2022-09-02T08:04:49.518493029,FloorSensor,sens3_A
4,2022-09-02,2022-09-02T08:04:55.554026934,FloorSensor,sens4_A
...,...,...,...,...
5814,2022-12-10,2022-12-10T08:37:04.449572377,FloorSensor,sens2_D
5815,2022-12-10,2022-12-10T08:37:34.449572377,FloorSensor,sens2_D
5816,2022-12-10,2022-12-10T08:38:04.449572377,FloorSensor,sens2_D
5817,2022-12-10,2022-12-10T08:38:34.449572377,FloorSensor,sens2_D


In [3]:
newdataset = pd.DataFrame(columns =["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
temp = 0
i = 0

for index, row in senseData.iterrows():
    activity = row["Sensor"][-1]
    new_row = {}
    new_row["Case_ID"] = row["Case_ID"]
    new_row["Timestamp"] = row["Timestamp"]
    new_row["Type"] = row["Type"]
    new_row["Sensor"] = row["Sensor"]
    # First row of the dataset
    if(temp == 0 and i == 0):
        new_row["Label"] = activity
        new_row["Activity_ID"] = i
        temp = activity
    # if the actual row has the same label of the stored, then is the same group
    elif(activity == temp):
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    # if the label mismatch, start a new group
    elif(activity != temp):
        i = i+1
        temp = activity
        new_row["Activity_ID"] = i
        new_row["Label"] = activity
    newdataset = newdataset.append(new_row, ignore_index=True)
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02T08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02T08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02T08:04:19.518493029,FloorSensor,sens3_A,A,0
3,2022-09-02,2022-09-02T08:04:49.518493029,FloorSensor,sens3_A,A,0
4,2022-09-02,2022-09-02T08:04:55.554026934,FloorSensor,sens4_A,A,0
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10T08:37:04.449572377,FloorSensor,sens2_D,D,399
5815,2022-12-10,2022-12-10T08:37:34.449572377,FloorSensor,sens2_D,D,399
5816,2022-12-10,2022-12-10T08:38:04.449572377,FloorSensor,sens2_D,D,399
5817,2022-12-10,2022-12-10T08:38:34.449572377,FloorSensor,sens2_D,D,399


In [4]:
# Convert the date columns in datetime type

newdataset['Timestamp'] = pd.to_datetime(newdataset['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f")
newdataset

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02 08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02 08:04:19.518493029,FloorSensor,sens3_A,A,0
3,2022-09-02,2022-09-02 08:04:49.518493029,FloorSensor,sens3_A,A,0
4,2022-09-02,2022-09-02 08:04:55.554026934,FloorSensor,sens4_A,A,0
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10 08:37:04.449572377,FloorSensor,sens2_D,D,399
5815,2022-12-10,2022-12-10 08:37:34.449572377,FloorSensor,sens2_D,D,399
5816,2022-12-10,2022-12-10 08:38:04.449572377,FloorSensor,sens2_D,D,399
5817,2022-12-10,2022-12-10 08:38:34.449572377,FloorSensor,sens2_D,D,399


In [5]:
out = pd.DataFrame(columns = ["Case_ID", "Timestamp", "Type", "Sensor", "Label", "Activity_ID"])
i = 0
for id, window in newdataset.groupby(pd.Grouper(freq="30s", key = "Timestamp")):
    if(window.empty):
        pass
    else:
        most_freq_label = window['Label'].value_counts().idxmax()
        dict = {"Case_ID":[],"Timestamp":[],"Type":[],"Sensor":[],"Label":[],"Activity_ID":[]}
        for a, el in window.iterrows():
            dict.get("Case_ID").append(el.get("Case_ID"))
            dict.get("Timestamp").append(el.get("Timestamp"))
            dict.get("Type").append(el.get("Type"))
            dict.get("Sensor").append(el.get("Sensor"))
            dict.get("Label").append(most_freq_label)
            dict.get("Activity_ID").append(i)
        i = i+1
        df2 = pd.DataFrame(dict)
        out = pd.concat([out, df2], ignore_index = True)
        # if(i==20):
        #     break

In [6]:
out

,Case_ID,Timestamp,Type,Sensor,Label,Activity_ID
0,2022-09-02,2022-09-02 08:03:44.275852343,PresenceSensor,door_A,A,0
1,2022-09-02,2022-09-02 08:03:49.518493029,FloorSensor,sens3_A,A,0
2,2022-09-02,2022-09-02 08:04:19.518493029,FloorSensor,sens3_A,A,1
3,2022-09-02,2022-09-02 08:04:49.518493029,FloorSensor,sens3_A,A,2
4,2022-09-02,2022-09-02 08:04:55.554026934,FloorSensor,sens4_A,A,2
...,...,...,...,...,...,...
5814,2022-12-10,2022-12-10 08:37:04.449572377,FloorSensor,sens2_D,D,4779
5815,2022-12-10,2022-12-10 08:37:34.449572377,FloorSensor,sens2_D,D,4780
5816,2022-12-10,2022-12-10 08:38:04.449572377,FloorSensor,sens2_D,D,4781
5817,2022-12-10,2022-12-10 08:38:34.449572377,FloorSensor,sens2_D,D,4782


In [7]:
# Save the file
out.to_csv("Ltrain_windowed_labeled.csv", index=False)

In [8]:
columns = ["Timestamp"] + out.Sensor.unique().tolist() + ["Label"]
labeled_senseData = pd.DataFrame(columns=columns)

In [11]:
# Transpose the log and create a row for each activation, incrementing the counter each time
grouped = out.groupby("Activity_ID")
i = 0
for a, group in grouped:
    group = group.reset_index()
    counter = {}
    for ind, el in group.iterrows():
        newrow = {}
        newrow["Timestamp"] = el["Timestamp"]
        newrow["Label"] = el["Label"]
        # print(counter)
        if el["Sensor"] in counter:
            counter[el["Sensor"]] = counter[el["Sensor"]]+1
        else:
            counter[el["Sensor"]] =1
        for k, v in counter.items():
            newrow[k] = v
        tempp = pd.DataFrame([newrow])
        labeled_senseData = pd.concat([labeled_senseData, tempp], ignore_index=True)
    i +=1
    # if(i==2):
    #      break
# print(labeled_senseData)
labeled_senseData

In [10]:
# Save the file
labeled_senseData.to_csv("Ltrain_labeled_trans.csv", index=False)